In [1]:
import numpy as np
import dataloader
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import models
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import os
import time
from torch_geometric.nn import GATv2Conv, global_mean_pool
# reload library
import importlib
import cv2
#import utils as ut
import pandas as pd
import DataDLC
from torch_geometric.data import Data, DataLoader
import tqdm


# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

## Analyse one video for Dominance.

In [2]:
def save_checkpoint(model, optimizer, epoch, loss, path):
    # Save the model, optimizer state, epoch, and loss
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, path)
    print(f"Checkpoint saved at {path}")

# Load the model
def load_checkpoint(model, optimizer, path, device):
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint loaded from {path}, at epoch {epoch}")
    return model, optimizer, epoch

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
importlib.reload(models)
# Define the model
graphencoder = models.GATEncoder(nout = 8, nhid=16, attention_hidden=2, n_in=4, dropout=0.5)
class_head = models.ClassificationHead(n_latent=576, nhid = 32, nout = 2)

model = models.GraphClassifier(graphencoder, class_head)

model.to(device)


GraphClassifier(
  (encoder): GATEncoder(
    (relu): ReLU()
    (gatenc1): GATv2Conv(4, 16, heads=2)
    (gatenc2): GATv2Conv(32, 16, heads=2)
    (gatenc3): GATv2Conv(32, 16, heads=2)
    (res_conn): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): ReLU()
    )
  )
  (classifier): ClassificationHead(
    (hidden1): Linear(in_features=576, out_features=32, bias=True)
    (hidden2): Linear(in_features=32, out_features=32, bias=True)
    (hidden3): Linear(in_features=32, out_features=2, bias=True)
    (relu): ReLU()
  )
)

In [5]:
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)

In [6]:
# Load the model
checkpoint_path = r'c:\Users\jalvarez\Documents\Data\Checkpoints\Dominance\checkpoint_epoch_400.pth'
model, optimizer, start_epoch = load_checkpoint(model, optimizer, checkpoint_path, device)

Checkpoint loaded from c:\Users\jalvarez\Documents\Data\Checkpoints\Dominance\checkpoint_epoch_400.pth, at epoch 200


#### Create dataloader

In [85]:
import warnings
warnings.filterwarnings("ignore")

data_loader = dataloader.DLCDataLoader(r'c:\Users\jalvarez\Documents\Data\OneVideoToAnalyse\dvvdvd', load_dataset=False, batch_size=1, window_size=5, stride=1, build_graph = True)


['MDXCV_mal_Test_16DLC_dlcrnetms5_More_BodyPartsJul9shuffle1_740000_el_filtered.h5']
Loading data from c:\Users\jalvarez\Documents\Data\OneVideoToAnalyse\dvvdvd, where we have 1 files
We have 1 files
Loading file MDXCV_mal_Test_16DLC_dlcrnetms5_More_BodyPartsJul9shuffle1_740000_el_filtered.h5


100%|██████████| 2614/2614 [00:37<00:00, 69.67it/s]

Number of files: 1


In [86]:
dataset = data_loader.data_list
len(dataset)

2614

In [87]:
dataset[0].behaviour

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [88]:
dataset[0].behaviour_names

Index(['General_Contacts', 'Sniffing_R', 'Sniffing_head_R', 'Sniffing_other_R',
       'Sniffing_anal_R', 'Poursuit_R', 'Dominance_R', 'Rearing_R',
       'Grooming_R', 'Sniffing_V', 'Sniffing_head_V', 'Sniffing_other_V',
       'Sniffing_anal_V', 'Poursuit_V', 'Dominance_V', 'Rearing_V',
       'Grooming_V'],
      dtype='object')

In [89]:
# Keep Only Dominance
for i in range(len(dataset)):
    dataset[i].behaviour = dataset[i].behaviour[6]

In [90]:
dataset[0].behaviour

tensor(0)

In [91]:
data = DataLoader(dataset, batch_size=1, shuffle=False)

In [92]:
len(data)

2614

In [93]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for graph in tqdm.tqdm(data):
        graph = graph.to(device)
        out = model(graph)
        y_true.append(graph.behaviour)
        y_pred.append(out.argmax().item())
        

100%|██████████| 2614/2614 [00:22<00:00, 114.38it/s]


In [94]:
# Accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))


0.8787299158377965


In [95]:
# Save the results
false_positives = np.zeros(len(y_true))
false_negatives = np.zeros(len(y_true))

for i in range(len(y_true)):
    if y_true[i] != y_pred[i]:
        if y_true[i] == 1:
            false_negatives[i] = 1
        else:
            false_positives[i] = 1
            

In [96]:
print(f"False Positives: {np.sum(false_positives)}")
print(f"False Negatives: {np.sum(false_negatives)}")
print(f'Real Positives: {np.sum(y_true)}')
print(f'Detected Positives: {np.sum(y_pred)}')


False Positives: 289.0
False Negatives: 28.0
Real Positives: 51
Detected Positives: 312


In [97]:
y_true = np.array(y_true).squeeze()

In [98]:
# Save the results
results = pd.DataFrame({'Frames': np.arange(len(y_true)), 'True': y_true, 'Predicted': y_pred, 'FalsePositive': false_positives, 'FalseNegative': false_negatives})

In [99]:
results

,Frames,True,Predicted,FalsePositive,FalseNegative
0,0,0,0,0.0,0.0
1,1,0,0,0.0,0.0
2,2,0,0,0.0,0.0
3,3,0,0,0.0,0.0
4,4,0,0,0.0,0.0
...,...,...,...,...,...
2609,2609,0,1,1.0,0.0
2610,2610,0,0,0.0,0.0
2611,2611,0,1,1.0,0.0
2612,2612,0,0,0.0,0.0


In [69]:
# make directory
path = r'c:\Users\jalvarez\Documents\Data\OneVideoToAnalyse\MDXCV_mal_Test_16'
if not os.path.exists(path):
    os.makedirs(path)

In [70]:
results.to_csv(r'c:\Users\jalvarez\Documents\Data\OneVideoToAnalyse\MDXCV_mal_Test_16\resutls.csv', index=False)